In [1]:
import pandas as pd
import numpy as np
import pickle
import joblib

from tkinter import filedialog
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn import metrics
from sklearn.metrics import classification_report

In [9]:
file1 = filedialog.askopenfilename()
rawdata = pd.read_csv(file1, sep=",",  skip_blank_lines=True)#inputfile
rawdata

,Unnamed: 0,Acceleration_in_X,Acceleration_in_Y,Acceleration_in_Z,Roll,Pitch,Yaw,Lane_change,turning,braking,acceleration
0,0,-0.005,0.008,0.018,-1.523,0.015,0.012,0.0,0.0,0.0,0.0
1,1,0.016,-0.002,0.018,-1.522,0.012,0.012,0.0,0.0,0.0,0.0
2,2,0.037,-0.005,0.018,-1.520,0.014,0.011,0.0,0.0,0.0,0.0
3,3,0.038,-0.009,0.024,-1.523,0.014,0.011,0.0,0.0,0.0,0.0
4,4,0.012,-0.016,0.032,-1.525,0.012,0.011,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
301526,301526,0.012,-0.008,0.046,-1.486,-0.027,1.616,0.0,0.0,1.0,0.0
301527,301527,0.018,-0.007,0.038,-1.488,-0.028,1.613,0.0,0.0,1.0,0.0
301528,301528,0.010,-0.001,0.029,-1.490,-0.031,1.610,0.0,0.0,1.0,0.0
301529,301529,-0.007,0.005,0.025,-1.491,-0.033,1.608,0.0,0.0,1.0,0.0


In [23]:
driveraw = rawdata.to_numpy()#choose six feature
drivedata = driveraw[:, 2:7]
drivedata

array([[ 8.000e-03,  1.800e-02, -1.523e+00,  1.500e-02,  1.200e-02],
       [-2.000e-03,  1.800e-02, -1.522e+00,  1.200e-02,  1.200e-02],
       [-5.000e-03,  1.800e-02, -1.520e+00,  1.400e-02,  1.100e-02],
       ...,
       [-1.000e-03,  2.900e-02, -1.490e+00, -3.100e-02,  1.610e+00],
       [ 5.000e-03,  2.500e-02, -1.491e+00, -3.300e-02,  1.608e+00],
       [ 1.500e-02,  1.900e-02, -1.494e+00, -3.400e-02,  1.605e+00]])

In [11]:
drivelabel = driveraw[:, 7]#choose label
drivelabel.astype(int)
drivelabel

array([0., 0., 0., ..., 0., 0., 0.])

In [12]:
lanchange_model = joblib.load('lanchange_model')

In [13]:
lanchange_predict = lanchange_model.predict(drivedata)

In [14]:
answer = []
for i in range(len(lanchange_predict)):
    if -2.5 < lanchange_predict[i] < -1.5:
        answer.append(-2)
    elif -1.5 < lanchange_predict[i] < -0.5:
        answer.append(-1)
    elif -0.5 < lanchange_predict[i] < 0.5:
        answer.append(0)
    elif 0.5 < lanchange_predict[i] < 1.5:
        answer.append(1)
    elif 1.5 < lanchange_predict[i] < 2.5:
        answer.append(2)

In [15]:
metrics.accuracy_score(drivelabel,answer)

1.0

In [4]:
breaking_model = joblib.load('breaking_model')

In [5]:
a = breaking_predict = breaking_model.predict([[-0.007,0.005,0.025,-1.491,-0.033,1.608]])
a

array([1.])

In [18]:
breaking_answer = []
for i in range(len(breaking_predict)):
    if breaking_predict[i] <= 0.75:
        breaking_answer.append(0)
    elif 0.75 < breaking_predict[i] <= 1.5:
        breaking_answer.append(1)
    elif 1.5 < breaking_predict[i] <= 2.25:
        breaking_answer.append(2)
    elif 2.25 < breaking_predict[i] <= 3:
        breaking_answer.append(3)

In [19]:
breaking_label = driveraw[:, 9]#choose label
breaking_label.astype(int)
metrics.accuracy_score(breaking_label, breaking_answer)

0.9493849720260935

In [20]:
report = classification_report(breaking_label, breaking_predict, digits=4)#precision and recall
print(report)

C:\Users\gordon_zhuang\Anaconda3\envs\forjupyter\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gordon_zhuang\Anaconda3\envs\forjupyter\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        -2.0     0.0000    0.0000    0.0000         0
        -1.0     0.0000    0.0000    0.0000         0
         0.0     0.9589    0.9821    0.9704    289112
         1.0     0.0518    0.0126    0.0203      9366
         2.0     0.0000    0.0000    0.0000      2903
         3.0     0.0000    0.0000    0.0000       150

    accuracy                         0.9421    301531
   macro avg     0.1684    0.1658    0.1651    301531
weighted avg     0.9210    0.9421    0.9310    301531



In [21]:
langchange_randomforest_model = joblib.load('langchange_randomforest_model')

In [24]:
lanoredict = langchange_randomforest_model.predict(drivedata)

In [25]:
report = classification_report(drivelabel, lanoredict, digits=5)#precision and recall
print(report)

              precision    recall  f1-score   support

        -2.0    1.00000   0.86299   0.92646       635
        -1.0    0.99803   0.91802   0.95636      1659
         0.0    0.99816   0.99998   0.99907    296132
         1.0    0.99902   0.89328   0.94319      2277
         2.0    0.99733   0.90338   0.94804       828

    accuracy                        0.99817    301531
   macro avg    0.99851   0.91553   0.95462    301531
weighted avg    0.99817   0.99817   0.99812    301531



In [26]:
xgb_lanechange_model = joblib.load('xgb_lanechange_model')

In [27]:
xgb_lane_predict = xgb_lanechange_model.predict(drivedata)

In [28]:
report = classification_report(drivelabel, xgb_lane_predict, digits=5)#precision and recall
print(report)

              precision    recall  f1-score   support

        -2.0    0.98480   0.91811   0.95029       635
        -1.0    0.99616   0.93731   0.96584      1659
         0.0    0.99870   0.99986   0.99928    296132
         1.0    0.99524   0.91875   0.95547      2277
         2.0    0.97750   0.94444   0.96069       828

    accuracy                        0.99858    301531
   macro avg    0.99048   0.94370   0.96631    301531
weighted avg    0.99857   0.99858   0.99856    301531

